In [16]:
import sqlite3
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [17]:
class Futball():
    
    def __init__(self, database, year):
        self.database = database
        self.year = year
    
    def total_goal(self):
        conn = sqlite3.Connection(self.database)
        c = conn.cursor()

        c.execute('''select m.HomeTeam as Team, Sum(FTHG) as Total_Home_Goal
                    from Matches m
                    where Season = ''' + self.year +
                    ''' 
                    group by m.HomeTeam
                    order by HomeTeam
                    ''')
        df_home_goals = pd.DataFrame(c.fetchall())
        df_home_goals.columns = [x[0] for x in c.description]

        c.execute('''select m.AwayTeam as Team, Sum(FTAG) as Total_Away_Goal
                    from Matches m
                    where Season = ''' + self.year +
                    '''
                    group by m.AwayTeam
                    order by m.AwayTeam
                    ''')
        df_away_goals = pd.DataFrame(c.fetchall())
        df_away_goals.columns = [x[0] for x in c.description]

        df_away_goals['Total']= df_home_goals['Total_Home_Goal']+df_away_goals['Total_Away_Goal']
        df_away_goals.drop('Total_Away_Goal', axis =1, inplace = True)
        
        return df_away_goals
    
    def total_win(self):
        conn = sqlite3.Connection(self.database)
        c = conn.cursor()
        
        c.execute('''select m.HomeTeam as Team, count(FTR) as HomeWin
            from Matches m
            where Season = ''' + self.year +
                    ''' and FTR = 'H'
            group by Team
            order by Team
            ''')
        df_home_win = pd.DataFrame(c.fetchall())
        df_home_win.columns = [x[0] for x in c.description]
        
        
        c.execute('''select m.AwayTeam as Team, count(FTR) as AwayWin
            from Matches m
            where Season = ''' + self.year +
                    ''' and FTR = 'A'
            group by Team
            order by Team
            ''')
        df_away_win = pd.DataFrame(c.fetchall())
        df_away_win.columns = [x[0] for x in c.description]
        
        df_away_win['Total Win']= df_home_win['HomeWin']+df_away_win['AwayWin']
        df_away_win.drop('AwayWin', axis =1, inplace = True)
        return df_away_win
    
    def total_lose(self):
        conn = sqlite3.Connection(self.database)
        c = conn.cursor()
        
        c.execute('''select m.HomeTeam as Team, count(FTR) as HomeLose
            from Matches m
            where Season = ''' + self.year +
                    ''' and FTR = 'A'
            group by Team
            order by Team
            ''')
        df_home_lose = pd.DataFrame(c.fetchall())
        df_home_lose.columns = [x[0] for x in c.description]
        
        
        c.execute('''select m.AwayTeam as Team, count(FTR) as AwayLose
            from Matches m
            where Season = ''' + self.year +
                    ''' and FTR = 'H'
            group by Team
            order by Team
            ''')
        df_away_lose = pd.DataFrame(c.fetchall())
        df_away_lose.columns = [x[0] for x in c.description]
        listt = []
        for i in df_away_lose['Team']:
            if i in list(df_home_lose['Team']):
                continue
            else:
                listt.append([i,0])
        dff = pd.DataFrame(listt,columns = ['Team', 'HomeLose'])
        df_home_lose = df_home_lose.append(dff)
        df_home_lose.sort_values(by=['Team'])
        list_homelose = list(df_home_lose['HomeLose'])
        for i in range(len(df_away_lose)):
            df_away_lose['AwayLose'][i]+=list_homelose[i]
        df_away_lose.rename(index=str, columns={"AwayLose": "Total Loss"}, inplace=True)
        return df_away_lose




        



In [18]:
g1 = Futball('database.sqlite', '2011')

In [19]:
g1.total_lose()

,Team,Total Loss
0,Aachen,15
1,Arsenal,10
2,Aston Villa,14
3,Augsburg,12
4,Bayern Munich,7
5,Blackburn,23
6,Bochum,17
7,Bolton,22
8,Braunschweig,9
9,Chelsea,10


In [246]:
c.execute('''select m.HomeTeam as Team, count(FTR) as HomeLose
            from Matches m
            where Season = '2011' and FTR = 'A'
            group by Team
            order by Team
            ''')
df_home_lose = pd.DataFrame(c.fetchall())
df_home_lose.columns = [x[0] for x in c.description]


c.execute('''select m.AwayTeam as Team, count(FTR) as AwayLose
    from Matches m
    where Season = '2011' and FTR = 'H'
    group by Team
    order by Team
    ''')
df_away_lose = pd.DataFrame(c.fetchall())
df_away_lose.columns = [x[0] for x in c.description]






In [247]:
listt = []
for i in df_away_lose['Team']:
    if i in list(df_home_lose['Team']):
        continue
    else:
        listt.append([i,0])
        
        

In [248]:
dff = pd.DataFrame(listt,columns = ['Team', 'HomeLose'])
df_home_lose = df_home_lose.append(dff)

,Team,HomeLose
0,Aachen,7
1,Arsenal,3
2,Aston Villa,8
3,Augsburg,4
4,Bayern Munich,2
5,Blackburn,12
6,Bochum,7
7,Bolton,11
8,Braunschweig,3
9,Chelsea,4


In [253]:
df_home_lose.sort_values(by=['Team']).reset_index().drop('index', axis = 1)

,Team,HomeLose
0,Aachen,7
1,Arsenal,3
2,Aston Villa,8
3,Augsburg,4
4,Bayern Munich,2
5,Blackburn,12
6,Bochum,7
7,Bolton,11
8,Braunschweig,3
9,Chelsea,4


In [257]:
list_homelose = list(df_home_lose['HomeLose'])

In [261]:
for i in range(len(df_away_lose)):
    df_away_lose['AwayLose'][i]+=list_homelose[i]


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [263]:
df_away_lose.rename(index=str, columns={"AwayLose": "Total Loss"})


,Team,Total Loss
0,Aachen,15
1,Arsenal,10
2,Aston Villa,14
3,Augsburg,12
4,Bayern Munich,7
5,Blackburn,23
6,Bochum,17
7,Bolton,22
8,Braunschweig,9
9,Chelsea,10


In [172]:
g1 = Futball('database.sqlite', '2011')


In [148]:
g1.total_win()

,Team,Total Win
0,Aachen,6
1,Arsenal,21
2,Aston Villa,7
3,Augsburg,8
4,Bayern Munich,23
5,Blackburn,8
6,Bochum,10
7,Bolton,10
8,Braunschweig,10
9,Chelsea,18


In [129]:
c.execute('''select m.HomeTeam as Team, count(FTR) as HomeWin
            from Matches m
            where Season = "2011" and FTR = 'H'
            group by Team
            order by Team
            ''')
df_home_win = pd.DataFrame(c.fetchall())
df_home_win.columns = [x[0] for x in c.description]
df_home_win

,Team,HomeWin
0,Aachen,4
1,Arsenal,12
2,Aston Villa,4
3,Augsburg,6
4,Bayern Munich,14
5,Blackburn,6
6,Bochum,7
7,Bolton,4
8,Braunschweig,6
9,Chelsea,12


In [128]:
c.execute('''select m.AwayTeam as Team, count(FTR) as AwayWin
            from Matches m
            where Season = "2011" and FTR = 'A'
            group by Team
            order by Team
            ''')
df_away_win = pd.DataFrame(c.fetchall())
df_away_win.columns = [x[0] for x in c.description]
df_away_win

,Team,AwayWin
0,Aachen,2
1,Arsenal,9
2,Aston Villa,3
3,Augsburg,2
4,Bayern Munich,9
5,Blackburn,2
6,Bochum,3
7,Bolton,6
8,Braunschweig,4
9,Chelsea,6


In [149]:
df_away_win['Total']=df_away_win['AwayWin']+df_home_win['HomeWin']

In [151]:
df_away_win.drop('AwayWin', axis=1)

,Team,Total
0,Aachen,6
1,Arsenal,21
2,Aston Villa,7
3,Augsburg,8
4,Bayern Munich,23
5,Blackburn,8
6,Bochum,10
7,Bolton,10
8,Braunschweig,10
9,Chelsea,18
